In [ ]:
%reload_ext autoreload
%autoreload 2
import os,sys
sys.path.insert(1, os.path.join(sys.path[0], '..', 'module'))
import wiki
import numpy as np
import pandas as pd
import networkx as nx
import scipy as sp

### Condensed sparse column matrices

In [ ]:
data = np.array([1, 2, 3, 4, 5, 6])
row = np.array([0, 2, 2, 0, 1, 2])
col = np.array([0, 0, 1, 2, 2, 2])
sp.sparse.csc_matrix((data, (row, col)), shape=(3, 3)).toarray()

In [ ]:
# topics = ['anatomy', 'biochemistry', 'cognitive science', 'evolutionary biology',
#           'genetics', 'immunology', 'molecular biology', 'chemistry', 'biophysics',
#           'energy', 'optics', 'earth science', 'geology', 'meteorology']
topics = ['earth science']

In [ ]:
path_saved = '/Users/harangju/Developer/data/wiki/graphs/dated/'
networks = {}
for topic in topics:
    print(topic, end=' ')
    networks[topic] = wiki.Net()
    networks[topic].load_graph(path_saved + topic + '.pickle')

In [ ]:
len(networks[topic].graph.nodes)

In [ ]:
v = networks[topic].graph.graph['tfidf']
v

In [ ]:
v.sum()

In [ ]:
v[:,0].indices[:5]

In [ ]:
v[4,0]

In [ ]:
networks[topic].graph.name

In [ ]:
networks[topic].graph.nodes['Biology']

In [ ]:
core = [n for n in networks[topic].graph.nodes if networks[topic].graph.nodes[n]['core_rb']>.9]
core

In [ ]:
[(i,n) for i,n in enumerate(networks[topic].graph.nodes) if networks[topic].graph.nodes[n]['year']<-1800]

In [ ]:
vi = v[:,9]
vi

### Distributions

In [ ]:
list(networks[topic].graph.successors('Cryosphere'))

In [ ]:
list(networks[topic].graph.predecessors('Cryosphere'))

In [ ]:
graph = networks[topic].graph
year_diffs = [[graph.nodes[node]['year'] - graph.nodes[neighbor]['year']
               for neighbor in list(graph.successors(node))]# + list(graph.predecessors(node))]
              for node in graph.nodes]

In [ ]:
year_diffs[0]

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

In [ ]:
sns.distplot([y for ys in year_diffs for y in ys])
plt.title(topic)
plt.xlabel('year difference');

In [ ]:
import sklearn.metrics.pairwise as smp

In [ ]:
similarities = [[smp.cosine_similarity(v[:,list(graph.nodes).index(node)].transpose(),
                                       v[:,list(graph.nodes).index(neighbor)].transpose())[0,0]
                 for neighbor in list(graph.successors(node))
                 if neighbor is not node]# + list(graph.predecessors(node))]
                for node in graph.nodes]

In [ ]:
similarities[0]

In [ ]:
from scipy.stats import norm

In [ ]:
mu, std = norm.fit([s for ss in similarities for s in ss])

In [ ]:
sns.distplot([s for ss in similarities for s in ss])
xmin, xmax = plt.xlim()
x = np.linspace(xmin, xmax, 100)
p = norm.pdf(x, mu, std)
plt.plot(x, p, 'k', linewidth=2)
print("Fit results: mu = %.2f,  std = %.2f" % (mu, std))
plt.title(topic)
plt.xlabel('cos similarity');

### CSC & networkx operations

In [ ]:
core = [n for n in networks[topic].graph.nodes if networks[topic].graph.nodes[n]['year']<-2000]
subgraph = graph.subgraph(core).copy()

In [ ]:
import scipy.sparse as ss

In [ ]:
tfidf = ss.hstack([v[:,list(graph.nodes).index(n)] for n in subgraph.nodes])
tfidf

In [ ]:
subgraph.nodes

In [ ]:
subgraph.add_node('Hello')

In [ ]:
subgraph.nodes

### Algorithm

Initialize with core set of nodes.\
For each year,\
initialize an "baby" node for each existing node that doesn't already have a baby node,\
mutate tf-idf for each "baby" node (including the name),\
and if the "baby" node gets a probability drawn from the distribution of similarities (to what?),
the "baby" node is born.

In [ ]:
import scipy.sparse as ss

def algorithm(graph):
    """
    Assumes graph.graph['tfidf'] holds a scipy.sparse.csc_matrix of tf-idf vectors.
    """
    core = [n for n in graph.nodes if graph.nodes[n]['year'] < -2000]
    tfidf = ss.hstack([graph.graph['tfidf'][:,list(graph.nodes).index(n)] for n in core])
    subgraph = graph.subgraph(core).copy()
    new_graph, new_tfidf = evolve(subgraph, tfidf)
    new_graph.graph['tfidf'] = new_tfidf
    return new_graph

def evolve(graph, tfidf, year_end=2020):
    year_start = max([graph.nodes[n]['year'] for n in graph.nodes])
    for year in range(year_start+1, year_end+1):
        mutate()
    return graph, tfidf

def mutate():
    pass

algorithm(networks[topic].graph)